This notebook generates recall and precision scores (for 0.5IOU) on the test grocery dataset with the model trained in `Colabs/GroceryDataset_Model_Training.ipynb` notebook. 

## Initial setup

In [1]:
# Install TFOD API (TF 1)
%tensorflow_version 1.x
import tensorflow as tf 
print(tf.__version__)

!git clone https://github.com/tensorflow/models.git

% cd models/research
!pip install --upgrade pip
# Compile protos.
!protoc object_detection/protos/*.proto --python_out=.
# Install TensorFlow Object Detection API.
!cp object_detection/packages/tf1/setup.py .
!python -m pip install --use-feature=2020-resolver .

TensorFlow 1.x selected.
1.15.2
Cloning into 'models'...
remote: Enumerating objects: 62654, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 62654 (delta 14), reused 46 (delta 12), pack-reused 62604
Receiving objects: 100% (62654/62654), 574.53 MiB | 31.82 MiB/s, done.
Resolving deltas: 100% (43649/43649), done.
/content/models/research
     |████████████████████████████████| 1.6 MB 15.2 MB/s 
  Attempting uninstall: pip
    Found existing installation: pip 21.1.3
    Uninstalling pip-21.1.3:
      Successfully uninstalled pip-21.1.3
Processing /content/models/research
  DEPRECATION: A future pip version will change local packages to be built in-place without first copying to a temporary directory. We recommend you use --use-feature=in-tree-build to test your packages with this new behavior before it becomes the default.
   pip 21.3 will remove support for this functionality. You can find discussion regarding this at ht

## Gather test record file, trained model graph and `.pbtxt`

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
!cp -r /content/drive/MyDrive/product-detection2/test.record .
!cp -r /content/drive/MyDrive/product-detection2/label_map.pbtxt .
!cp -r /content/drive/MyDrive/product-detection2/fp32/frozen_inference_graph.pb .

## Gather detections

In [5]:
# Infer detections on a TFRecord of TFExamples given an inference graph.
!python /content/models/research/object_detection/inference/infer_detections.py \
    --input_tfrecord_paths=test.record \
    --output_tfrecord_path=detections.tfrecord \
    --inference_graph=frozen_inference_graph.pb

2021-09-14 22:12:23.990482: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-14 22:12:23.990550: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ab49426b9cda): /proc/driver/nvidia/version does not exist
INFO:tensorflow:Reading input from 1 files
I0914 22:12:24.059536 139663807170432 infer_detections.py:68] Reading input from 1 files
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
W0914 22:12:24.060024 139663807170432 deprecation.py:345] From /usr/local/lib/python3.7/dist-packages/object_detection/inference/detection_inference.py:35: string_input_producer (from tensorflow.python.training.input) is deprecated and wil

## Generate metrics

In [6]:
!git clone https://github.com/svpino/tf_object_detection_cm

Cloning into 'tf_object_detection_cm'...
remote: Enumerating objects: 35, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 35 (delta 0), reused 3 (delta 0), pack-reused 31
Unpacking objects: 100% (35/35), done.


In [8]:
!python tf_object_detection_cm/confusion_matrix.py \
    --detections_record=detections.tfrecord \
    --label_map=label_map.pbtxt \
    --output_path=confusion_matrix.csv

2021-09-14 22:16:57.272102: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-09-14 22:16:57.272168: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ab49426b9cda): /proc/driver/nvidia/version does not exist
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
W0914 22:16:57.273885 140262804522880 deprecation.py:345] From tf_object_detection_cm/confusion_matrix.py:39: tf_record_iterator (from tensorflow.python.lib.io.tf_record) is deprecated and will be removed in a future version.
Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`
Processed 71 images

Confusion Matrix:
[[1.964e+03 0.000e+00 8.000e+00 0.000e+00 2.000e+00 1.000e+00 1.300e+01
  7.000e+00 3.000e+00 0.000e+00 1.000e+00 2.400e+01]
 [0.000e+00 4.700e+01 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
  0